In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

GOOGLE_CREDENTIAL_PATH = "/home/kuchoco/.google/credentials/google_credentials.json"
GCS_CONNECTOR_PATH ='/home/kuchoco/data_engineering/5.1_spark/lib/gcs-connector-hadoop3-latest.jar'


In [2]:
!gsutil -m cp -r data/ gs://dtc_data_lake_indigo-anchor-353504/from_local_spark

Copying file://data/report/green/.part-00015-88ebee63-e9f7-4598-8698-63e0251bc3e3-c000.snappy.parquet.crc [Content-Type=application/octet-stream]...
Copying file://data/report/green/.part-00001-88ebee63-e9f7-4598-8698-63e0251bc3e3-c000.snappy.parquet.crc [Content-Type=application/octet-stream]...
Copying file://data/report/green/part-00003-88ebee63-e9f7-4598-8698-63e0251bc3e3-c000.snappy.parquet [Content-Type=application/octet-stream]...
Copying file://data/report/green/.part-00011-88ebee63-e9f7-4598-8698-63e0251bc3e3-c000.snappy.parquet.crc [Content-Type=application/octet-stream]...
Copying file://data/report/green/part-00013-88ebee63-e9f7-4598-8698-63e0251bc3e3-c000.snappy.parquet [Content-Type=application/octet-stream]...
Copying file://data/report/green/.part-00006-88ebee63-e9f7-4598-8698-63e0251bc3e3-c000.snappy.parquet.crc [Content-Type=application/octet-stream]...
Copying file://data/report/green/._SUCCESS.crc [Content-Type=application/octet-stream]...
Copying file://data/report

In [3]:
!wget https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar -P lib

--2022-07-24 13:12:51--  https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar
Resolving storage.googleapis.com (storage.googleapis.com)... 34.64.4.16, 34.64.4.80, 34.64.4.48, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|34.64.4.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33831577 (32M) [application/java-archive]
Saving to: ‘lib/gcs-connector-hadoop3-latest.jar’

gcs-connector-hadoo 100%[===================>]  32.26M  8.62MB/s    in 4.0s    

2022-07-24 13:12:56 (8.14 MB/s) - ‘lib/gcs-connector-hadoop3-latest.jar’ saved [33831577/33831577]



In [2]:
conf = SparkConf()\
    .setMaster('local[*]')\
    .setAppName('gcs')\
    .set("spark.jars", GCS_CONNECTOR_PATH)\
    .set("spark.hadoop.google.cloud.auth.service.account.enable", 'true')\
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", GOOGLE_CREDENTIAL_PATH)

sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()
hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", GOOGLE_CREDENTIAL_PATH)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")


spark = SparkSession.builder\
    .config(conf=sc.getConf())\
    .getOrCreate()

22/07/24 22:32:19 WARN Utils: Your hostname, kuchoco-GP62-6QE resolves to a loopback address: 127.0.1.1; using 192.168.219.113 instead (on interface wlp2s0)
22/07/24 22:32:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/07/24 22:32:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df_green = spark.read.parquet('gs://dtc_data_lake_indigo-anchor-353504/from_local_spark/data/origin_green')

In [4]:
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2019-12-19 00:52:30|  2019-12-19 00:54:39|                 N|       1.0|         264|         264|            5.0|          0.0|        3.5|  0.5|    0.

In [5]:
df_green.count()

1734176